## Image Preparation

Requires that:
- malign.png be 3-channel
- normal.png be 3-channel
- malign_mask.png be one-channel (BINARY)

In [14]:
# Required imports
import sys
sys.path.append("../utils")
import os
import cv2
from utils import is_collage_possible, make_collage, make_3_channels_mask, get_latest_model, execute_bash_command, resize_to_dim

In [6]:
# Checks if collage is possible between two images
w, h = is_collage_possible(malign_mask_pth='../pipeline_data_tcia/benign/benign_mask.png', normal_breast_pth='../pipeline_data_tcia/normal/normal.png')
w, h

(1048, 997)

In [7]:
# If both width and height are different from -1 then collage is possible
if w != -1 and h != -1:
    make_collage(malign_pth='../pipeline_data_tcia/benign/benign.png', malign_mask_pth='../pipeline_data_tcia/benign/benign_mask.png', normal_pth='../pipeline_data_tcia/normal/normal.png', width=w, height=h)

In [10]:
# Make the collage mask 3-channel
make_3_channels_mask('collage_mask.png', 'collage_mask3.png')
os.remove('collage_mask.png')
os.rename('collage_mask3.png', 'collage_mask.png')

## Basic Harmonisation Training

In [11]:
# Change to correct directory
%cd ../MedSinGAN/

/Users/josedaviddomingues/Documents/Thesis/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN


In [ ]:
# Normal breast collage Harmonizer creation
! python main_train.py --train_mode harmonization --gpu 0 --train_stages 3 --im_min_size 120 --lrelu_alpha 0.3 --niter 1000 --batch_norm --input_name ../pipeline_data_tcia/normal/normal.png

In [ ]:
# Fine tunes the harmonisation model
m = get_latest_model("TrainedModels/normal")
fine_tune_cmd = "python main_train.py --gpu 0 --train_mode harmonization --input_name ../pipeline_data_tcia/normal/normal.png --fine_tune --model_dir " + str(m)

for path in execute_bash_command(fine_tune_cmd.split()):
    print(path, end="")

In [ ]:
# Harmonise the naive collage
m = get_latest_model()
harmonise_cmd = "python evaluate_model.py --gpu 0 --model_dir " + str(m) + " --naive_img ../notebooks/collage.png"
for path in execute_bash_command(harmonise_cmd.split()):
        print(path, end="")

In [ ]:
# Evaluate results
base_img = "../pipeline_data_tcia/normal/normal.png"
target_image = os.path.join(get_latest_model(),"Evaluation_", "content", "collage.png", "harmonized_w_mask.jpg")

im = cv2.imread(base_img)
resize_to_dim(target_image, im.shape[0], im.shape[1], "target_resized.png")

from evaluate_generation import GenerationEvaluator
evaluator = GenerationEvaluator(base_img)

lpips = evaluator.run_lpips_to_image("target_resized.png")
ssim, ms_ssim = evaluator.run_mssim_to_image("target_resized.png")
print(f"LPIPS: {lpips}\nSSIM: {ssim}\nMS-SSIM: {ms_ssim}")

## Combination Harmonisation Testing

In [21]:
# Combinations to test in the format below
# (N Stages, Concurrent Stages, Image Min Size, Image Max Size, L.Relu Alpha, N Iterations, Fine-Tune)
varying_min_combinations = [
    (3, 3, 120, 480, 0.3, 1000, True),
    (3, 3, 480, 1080, 0.3, 1000, True),
    (3, 3, 720, 1440, 0.3, 1000, True),
    (3, 3, 1080, 1920, 0.3, 1000, True),
    (3, 3, 1440, 2457, 0.3, 1000, True)]

varying_stages_combinations = [
    (3, 3, 120, 480, 0.3, 1000, True),
    (6, 3, 120, 1080, 0.3, 1000, True),
    (6, 3, 120, 1440, 0.3, 1000, True),
    (12, 6, 120, 1920, 0.3, 1000, True),
    (12, 6, 120, 2457, 0.3, 1000, True)]

In [18]:
# Execute single experiment with given parameters
def do_single_experiment(malign, malign_mask, normal, w, h, stages, min_i, max_i, alpha, niter, concurrent, fine_tune):
    
    make_collage(malign_pth=malign, malign_mask_pth=malign_mask, normal_pth=normal, width=w, height=h)
    make_3_channels_mask('collage_mask.png', 'collage_mask3.png')
    os.remove('collage_mask.png')
    os.rename('collage_mask3.png', 'collage_mask.png')
    %cd ../MedSinGAN/
    
    cmd = f"python main_train.py --train_mode harmonization --gpu 0 --train_stages {stages} --train_depth {concurrent} --im_min_size {min_i} --im_max_size {max_i} --lrelu_alpha {alpha} --niter {niter} --batch_norm --input_name {normal}"
    for path in execute_bash_command(cmd.split()):
        print(path, end="")
        
    if fine_tune:
        m = get_latest_model("TrainedModels/normal")
        fine_tune_cmd = f"python main_train.py --gpu 0 --train_mode harmonization --input_name {normal} --fine_tune --model_dir " + str(m)
        for path in execute_bash_command(fine_tune_cmd.split()):
            print(path, end="")
        
    m = get_latest_model()
    harmonise_cmd = "python evaluate_model.py --gpu 0 --model_dir " + str(m) + " --naive_img ../notebooks/collage.png"
    for path in execute_bash_command(harmonise_cmd.split()):
        print(path, end="")
        
    comand = f"zip -r ../S{stages}_MIN{min_i}MAX_{max_i}ITER_{niter}_CONC_{concurrent}.zip ."
    for path in execute_bash_command(comand.split()):
        print(path, end="")
        
    ! rm -r mlruns TrainedModels target_resized.png
    %cd ../notebooks/
    ! rm collage_mask.png collage.png malign_aux.png normal_aux.png

In [19]:
# Run experimentations
for comb in varying_stages_combinations:
    do_single_experiment('../pipeline_data_tcia/benign/benign.png', 
                         '../pipeline_data_tcia/benign/benign_mask.png',
                         '../pipeline_data_tcia/normal/normal.png', 
                         w, h, comb[0], comb[2], comb[3], comb[4], comb[5], comb[1], comb[6])
    
for comb in varying_min_combinations:
    do_single_experiment('../pipeline_data_tcia/benign/benign.png', 
                         '../pipeline_data_tcia/benign/benign_mask.png',
                         '../pipeline_data_tcia/normal/normal.png', 
                         w, h, comb[0], comb[2], comb[3], comb[4], comb[5], comb[1], comb[6])

/Users/josedaviddomingues/Documents/Thesis/Adversarial-Representation-Learning-for-Medical-Imaging/MedSinGAN


In [ ]:
# Run experimentations

## Save Model

In [ ]:
# Zip everything from this model
!zip -r current_model.zip .

## Delete Current Model

In [20]:
# Delete everything that the model produces
! rm -r mlruns TrainedModels target_resized.png
%cd ../notebooks/
! rm collage_mask.png collage.png malign_aux.png normal_aux.png

rm: mlruns: No such file or directory
rm: TrainedModels: No such file or directory
rm: target_resized.png: No such file or directory
/Users/josedaviddomingues/Documents/Thesis/Adversarial-Representation-Learning-for-Medical-Imaging/notebooks
rm: malign_aux.png: No such file or directory
rm: normal_aux.png: No such file or directory
